In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
!git clone https://github.com/DanielDimanov/nsga-net.git

Cloning into 'nsga-net'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 540 (delta 18), reused 15 (delta 7), pack-reused 509
Receiving objects: 100% (540/540), 2.50 MiB | 6.88 MiB/s, done.
Resolving deltas: 100% (255/255), done.


In [0]:
!cd nsga-net/

In [0]:
!pwd

/content


In [0]:
!pyt

/bin/bash: pyt: command not found


In [0]:
!ls nsga-net/

img  misc  models  README.md  search  validation  visualization


In [0]:
import sys
sys.path.append('/content/nsga-net/')

In [0]:
import search.cifar10_search as my_cifar10
import torchvision.transforms as transforms

In [0]:
CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

# data agumentation
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
train_transform.transforms.append(transforms.Normalize(CIFAR_MEAN, CIFAR_STD))

valid_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])

In [0]:
data_root = './data'
train_data = my_cifar10.CIFAR10(root=data_root, train=True, download=True, transform=train_transform)
valid_data = my_cifar10.CIFAR10(root=data_root, train=False, download=True, transform=valid_transform)

In [0]:
type(train_data)

search.cifar10_search.CIFAR10

In [0]:
import torch.utils.data as data
import pandas as pd
class WeaponDetection(data.Dataset):
    def __init__(self, labels_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(labels_file,sep=" ", header=None)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.landmarks_frame.ix[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.ix[idx, 1:].as_matrix().astype('float')
        landmarks = landmarks.reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [28]:
lblSFile="/content/gdrive/My Drive/datasetTrain/labelsFin.txt"
pd.read_csv(lblSFile,sep=" ",header=None)

,0
0,riffle
1,pistol
2,noThreat
3,knife


In [29]:
!unzip /content/gdrive/My\ Drive/MScProject/dataFinal.zip

Archive:  /content/gdrive/My Drive/MScProject/dataFinal.zip
replace dataFinal/train/riffle/riffle67.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataFinal/train/riffle/riffle209.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [30]:
from PIL import Image
import os
import pathlib
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

folders ={"train":"/content/dataFinal/train", "validation":"/content/dataFinal/validation"}
lblSFile="/content/gdrive/My Drive/datasetTrain/labelsFin.txt"
file = open(lblSFile, "r")
imCLs=[]
for imCL in file:
  imCLs.append(str(imCL)[:-1])
X_train,Y_train,X_val,Y_val=[],[],[],[]
for step in ['train','validation']:
    X,Y=[],[]
    for cl in imCLs:
        directory=''+folders[step]+"/"+str(cl)
        print("Getting images from:"+directory)
        size=512,512
        print("Processing class "+ str(cl))
        for filename in os.listdir(directory):
            if(filename.startswith('.')):
                continue
            else:
                img = image.load_img(directory+'/'+filename,target_size=(128,128,3),grayscale=False)
                img = image.img_to_array(img)
                img = img/255
                X.append(img)
                lbl=str(cl)
                Y.append(imCLs.index(lbl))
    if(step=='train'):
        X_train = np.array(X)
        Y_train = np.array(Y)
    if(step=='validation'):
        X_val = np.array(X)
        Y_val = np.array(Y)
    del X
    del Y

Using TensorFlow backend.


Getting images from:/content/dataFinal/train/riffle
Processing class riffle
Getting images from:/content/dataFinal/train/pistol
Processing class pistol
Getting images from:/content/dataFinal/train/noThreat
Processing class noThreat
Getting images from:/content/dataFinal/train/knife
Processing class knife
Getting images from:/content/dataFinal/validation/riffle
Processing class riffle
Getting images from:/content/dataFinal/validation/pistol
Processing class pistol
Getting images from:/content/dataFinal/validation/noThreat
Processing class noThreat
Getting images from:/content/dataFinal/validation/knife
Processing class knife


In [31]:
X_train_swap = np.swapaxes(X_train,-1,1)
X_val_swap = np.swapaxes(X_val,-1,1)
X_train_swap.shape

(2140, 3, 128, 128)

In [0]:
import torch

In [0]:
train_dataset = data.TensorDataset(torch.from_numpy(X_train_swap),torch.from_numpy(Y_train))
val_dataset = data.TensorDataset(torch.from_numpy(X_val_swap),torch.from_numpy(Y_val))

In [34]:
np.mean(X_train,axis=(0,1,2))

array([0.4785047 , 0.45566642, 0.42557254], dtype=float32)

In [35]:
np.std(X_train,axis=(0,1,2))

array([0.319471  , 0.31107742, 0.3122497 ], dtype=float32)

In [0]:
import os
import numpy as np
import torch
import logging
import argparse
import torch.nn as nn
import torch.utils
# import torchvision.datasets as dset
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

from models.macro_models import EvoNetwork
from models.micro_models import NetworkCIFAR as Network

import search.cifar10_search as my_cifar10

import time
from misc import utils
from search import micro_encoding
from search import macro_encoding
from misc.flops_counter import add_flops_counting_methods


device = 'cuda'


def main(genome, epochs, search_space='micro',
         save='Design_1', expr_root='search', seed=0, gpu=0, init_channels=24,
         layers=11, auxiliary=False, cutout=False, drop_path_prob=0.0):

    # ---- train logger ----------------- #
    save_pth = os.path.join(expr_root, '{}'.format(save))
    utils.create_exp_dir(save_pth)
    log_format = '%(asctime)s %(message)s'
    logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                        format=log_format, datefmt='%m/%d %I:%M:%S %p')
    # fh = logging.FileHandler(os.path.join(save_pth, 'log.txt'))
    # fh.setFormatter(logging.Formatter(log_format))
    # logging.getLogger().addHandler(fh)

    # ---- parameter values setting ----- #
    NUM_CLASSES = 4
    CIFAR_CLASSES = NUM_CLASSES
    DATA_SHAPE = (128, 128)
    INPUT_CHANNELS = 3
    learning_rate = 0.025
    momentum = 0.9
    weight_decay = 3e-4
    data_root = '../data'
    batch_size = 32
    cutout_length = 16
    auxiliary_weight = 0.4
    grad_clip = 5
    report_freq = 50
    train_params = {
        'auxiliary': auxiliary,
        'auxiliary_weight': auxiliary_weight,
        'grad_clip': grad_clip,
        'report_freq': report_freq,
    }

    if search_space == 'micro':
        genotype = micro_encoding.decode(genome)
        model = Network(init_channels, CIFAR_CLASSES, layers, auxiliary, genotype)
    elif search_space == 'macro':
        genotype = macro_encoding.decode(genome)
        channels = [(INPUT_CHANNELS, init_channels),
                    (init_channels, 2*init_channels),
                    (2*init_channels, 4*init_channels)]
        model = EvoNetwork(genotype, channels, CIFAR_CLASSES,DATA_SHAPE, decoder='residual')
    else:
        raise NameError('Unknown search space type')

    # logging.info("Genome = %s", genome)
    logging.info("Architecture = %s", genotype)

    torch.cuda.set_device(gpu)
    cudnn.benchmark = True
    torch.manual_seed(seed)
    cudnn.enabled = True
    torch.cuda.manual_seed(seed)

    n_params = (np.sum(np.prod(v.size()) for v in filter(lambda p: p.requires_grad, model.parameters())) / 1e6)
    model = model.to(device)

    logging.info("param size = %fMB", n_params)

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.cuda()

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.SGD(
        parameters,
        learning_rate,
        momentum=momentum,
        weight_decay=weight_decay
    )

    #TO DO: change
    CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
    DATASET_MEAN = [0.4785047 , 0.45649716, 0.42604172]
    CIFAR_MEAN = DATASET_MEAN
    DATASET_STD = [0.31962952, 0.3112294 , 0.31206125]
    CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]
    CIFAR_STD = DATASET_STD
#     # data agumentation
#     train_transform = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor()
#     ])

#     if cutout:
#         train_transform.transforms.append(utils.Cutout(cutout_length))

#     train_transform.transforms.append(transforms.Normalize(CIFAR_MEAN, CIFAR_STD))

#     valid_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
#     ])

#     train_data = my_cifar10.CIFAR10(root=data_root, train=True, download=True, transform=train_transform)
#     valid_data = my_cifar10.CIFAR10(root=data_root, train=False, download=True, transform=valid_transform)

#     # num_train = len(train_data)
#     # indices = list(range(num_train))
#     # split = int(np.floor(train_portion * num_train))
    train_data = train_dataset
    valid_data = val_dataset
    train_queue = torch.utils.data.DataLoader(
        train_data, batch_size=batch_size,
        # sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
        pin_memory=True, num_workers=4)

    valid_queue = torch.utils.data.DataLoader(
        valid_data, batch_size=batch_size,
        # sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:num_train]),
        pin_memory=True, num_workers=4)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, int(epochs))

    for epoch in range(epochs):
        scheduler.step()
        logging.info('epoch %d lr %e', epoch, scheduler.get_lr()[0])
        model.droprate = drop_path_prob * epoch / epochs

        train_acc, train_obj = train(train_queue, model, criterion, optimizer, train_params)
        logging.info('train_acc %f', train_acc)

    valid_acc, valid_obj = infer(valid_queue, model, criterion)
    logging.info('valid_acc %f', valid_acc)

    # calculate for flops
    model = add_flops_counting_methods(model)
    model.eval()
    model.start_flops_count()
    random_data = torch.randn(1, INPUT_CHANNELS, *DATA_SHAPE)
    model(torch.autograd.Variable(random_data).to(device))
    n_flops = np.round(model.compute_average_flops_cost() / 1e6, 4)
    logging.info('flops = %f', n_flops)

    # save to file
    # os.remove(os.path.join(save_pth, 'log.txt'))
    with open(os.path.join(save_pth, 'log.txt'), "w") as file:
        file.write("Genome = {}\n".format(genome))
        file.write("Architecture = {}\n".format(genotype))
        file.write("param size = {}MB\n".format(n_params))
        file.write("flops = {}MB\n".format(n_flops))
        file.write("valid_acc = {}\n".format(valid_acc))

    # logging.info("Architecture = %s", genotype))

    return {
        'valid_acc': valid_acc,
        'params': n_params,
        'flops': n_flops,
    }


# def train(train_queue, model, criterion, optimizer, params):
#     objs = utils.AvgrageMeter()
#     top1 = utils.AvgrageMeter()
#     top5 = utils.AvgrageMeter()
#     model.train()
#
#     for step, (input, target) in enumerate(train_queue):
#         input = Variable(input).cuda()
#         target = Variable(target).cuda(async=True)
#
#         optimizer.zero_grad()
#         if params['auxiliary']:
#             logits, logits_aux = model(input)
#         else:
#             logits, _ = model(input)
#
#         loss = criterion(logits, target)
#         if params['auxiliary']:
#             loss_aux = criterion(logits_aux, target)
#             loss += params['auxiliary_weight'] * loss_aux
#         loss.backward()
#         nn.utils.clip_grad_norm(model.parameters(), params['grad_clip'])
#         optimizer.step()
#
#         prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
#         n = input.size(0)
#         objs.update(loss.data[0], n)
#         top1.update(prec1.data[0], n)
#         top5.update(prec5.data[0], n)
#
#         # if step % params['report_freq'] == 0:
#         #     logging.info('train %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
#
#     return top1.avg, objs.avg

# Training
def train(train_queue, net, criterion, optimizer, params):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for step, (inputs, targets) in enumerate(train_queue):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs, outputs_aux = net(inputs)
        loss = criterion(outputs, targets)

        if params['auxiliary']:
            loss_aux = criterion(outputs_aux, targets)
            loss += params['auxiliary_weight'] * loss_aux

        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), params['grad_clip'])
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    #     if step % args.report_freq == 0:
    #         logging.info('train %03d %e %f', step, train_loss/total, 100.*correct/total)
    #
    # logging.info('train acc %f', 100. * correct / total)

    return 100.*correct/total, train_loss/total


# def infer(valid_queue, model, criterion):
#     objs = utils.AvgrageMeter()
#     top1 = utils.AvgrageMeter()
#     top5 = utils.AvgrageMeter()
#     model.eval()
#
#     for step, (input, target) in enumerate(valid_queue):
#         input = Variable(input, volatile=True).cuda()
#         target = Variable(target, volatile=True).cuda(async=True)
#
#         logits, _ = model(input)
#
#         loss = criterion(logits, target)
#
#         prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
#         n = input.size(0)
#         objs.update(loss.data[0], n)
#         top1.update(prec1.data[0], n)
#         top5.update(prec5.data[0], n)
#
#         # if step % params['report_freq'] == 0:
#         #     logging.info('valid %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
#
#     return top1.avg, objs.avg


def infer(valid_queue, net, criterion):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for step, (inputs, targets) in enumerate(valid_queue):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs, _ = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # if step % args.report_freq == 0:
            #     logging.info('valid %03d %e %f', step, test_loss/total, 100.*correct/total)

    acc = 100.*correct/total
    # logging.info('valid acc %f', 100. * correct / total)

    return acc, test_loss/total


In [0]:
!mkdir searchv2

In [41]:
    DARTS_V2 = [[[[3, 0], [3, 1]], [[3, 0], [3, 1]], [[3, 1], [2, 0]], [[2, 0], [5, 2]]],
               [[[0, 0], [0, 1]], [[2, 2], [0, 1]], [[0, 0], [2, 2]], [[2, 2], [0, 1]]]]
    start = time.time()
    print(main(genome=DARTS_V2, epochs=20, save='BATCH_TEST', seed=1, init_channels=16,
               auxiliary=False, cutout=False, drop_path_prob=0.0))
    print('Time elapsed = {} mins'.format((time.time() - start)/60))

Experiment dir : search/BATCH_TEST
48 48 16
48 48 16
48 48 16
48 48 32
48 96 32
96 96 32
96 96 32
96 96 64
96 192 64
192 192 64
192 192 64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


KeyboardInterrupt: ignored

In [0]:
torch.cuda.empty_cache() 

In [0]:
!nvidia-smi

Tue Aug  6 13:40:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    57W / 149W |    668MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!sudo nvidia-smi --gpu-reset -i 0

Failed to initialize NVML: Driver/library version mismatch


In [0]:
!git clone https://github.com/ianwhale/nsga-net.git nsga-net-official

Cloning into 'nsga-net-official'...
remote: Enumerating objects: 509, done.
remote: Total 509 (delta 0), reused 0 (delta 0), pack-reused 509
Receiving objects: 100% (509/509), 2.49 MiB | 1.95 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [0]:
!python search/evolution_search.py --search_space macro --init_channels 32 --n_gens 30

Traceback (most recent call last):
  File "search/evolution_search.py", line 9, in <module>
    from misc import utils
ModuleNotFoundError: No module named 'misc'


In [0]:
__init__.py file

import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

Traceback (most recent call last):
  File "validation/test.py", line 20, in <module>
    from misc import utils
ModuleNotFoundError: No module named 'misc'


In [0]:
!cat  __init__.py

import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

In [0]:
with open("__init__.py","w") as f:
  f.write("import os\n")
  f.write("import sys\n")
  f.write("sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))")

In [0]:
!touch ../gdrive/My\ Drive/__init__.py

In [0]:
!ls ../gdrive/My\ Drive

'2017-05-15 10-55 page 1.pdf'
 amoeba.json
'Applications of Nuclear Physics in Medicine (1) (1).gslides'
'Applications of Nuclear Physics in Medicine (1) (2).gslides'
'Applications of Nuclear Physics in Medicine (1).gslides'
 Article.gdoc
'article shit.docx'
 Black_sea_coast.pdf
 build_image_data.py
 build_image_dataTen.py
 Bulgarian_Diploma_Calculator.xls
'Burp Suite Professional v1.5.01-1.rar'
'Colab Notebooks'
 collab_data.gsheet
'Copy of UVH website ER'
'Customer Feedback.gform'
 Cv
'Daniel Dimanov.doc'
'Daniel Dimanov.doc.gdoc'
'Daniel DimanovPersonal Statement Edited.doc'
 datasetTrain
 debelqnov.doc
 Doc1.docx
 drawable
 dyasiudij.gdraw
'Final Projectium.gdoc'
 GeneticCNN_CIFAR10_AUGM.rar
'Geo East Europe.gdoc'
'gledam strashno.zip'
 IMG_0390.MOV
"Important Interview q's.docx"
 Installatron
 jpeg_to_tf_record_Danv2.py
 jpeg_to_tf_record.py
 KD
 Lease_Agreement_Bright_Buildings_Rutronik_anex_NB.docx
 Mama
'Map Projectum.pdf'
'Math Project.gform'
'Math Project (Responses).gsheet'


In [5]:
!pip install pymoo

     |████████████████████████████████| 532kB 5.3MB/s 
  Created wheel for pymoo: filename=pymoo-0.3.0-cp36-cp36m-linux_x86_64.whl size=1713415 sha256=136524aac47c1d204b0ce7da5b395861540921080e30fcd773190a2c16496df3
  Stored in directory: /root/.cache/pip/wheels/6a/ca/e9/4217337eac3d9894873ff103bfe69026427a6fe205af349952
Successfully built pymoo


In [0]:
!export PYTHONPATH="$PYTHONPATH:/content/nsga-net-official/"

In [0]:
import sys
sys.path.append('/content/nsga-net-official/')

In [0]:
sys.path[9]=sys.path[10]

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/nsga-net/',
 '/content/nsga-net-official/']

In [0]:
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/nsga-net-official/']

In [0]:
sys.path.remove('/content/nsga-net/')

In [13]:
!git clone https://github.com/DanielDimanov/nsga-net.git

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/DanielDimanov/nsga-net
 * branch            HEAD       -> FETCH_HEAD
Updating 55a831e..e95ee66
Fast-forward
 search/evolution_search.py | 65 +++++++++++++++++++++++++++++++++++++++++++++-
 search/train_search.py     |  2 +-
 2 files changed, 65 insertions(+), 2 deletions(-)


In [8]:
!python nsga-net/search/evolution_search.py --save NSGATESTv1 --search_space micro --init_channels 16 --layers 8 --epochs 3 --n_offspring 20 --n_gens 30

Experiment dir : search-NSGATESTv1-micro-20190808-000917
Using TensorFlow backend.
Getting images from:/content/dataFinal/train/riffle
Processing class riffle
Getting images from:/content/dataFinal/train/pistol
Processing class pistol
Getting images from:/content/dataFinal/train/noThreat
Processing class noThreat
Getting images from:/content/dataFinal/train/knife
Processing class knife
Getting images from:/content/dataFinal/validation/riffle
Processing class riffle
Getting images from:/content/dataFinal/validation/pistol
Processing class pistol
Getting images from:/content/dataFinal/validation/noThreat
Processing class noThreat
Getting images from:/content/dataFinal/validation/knife
Processing class knife
08/08 12:09:29 AM args = Namespace(epochs=3, init_channels=16, labels_file_path='/content/gdrive/My Drive/datasetTrain/labelsFin.txt', layers=8, n_blocks=5, n_cells=2, n_gens=30, n_nodes=4, n_offspring=20, n_ops=9, pop_size=40, save='search-NSGATESTv1-micro-20190808-000917', search_sp

In [0]:
X,Y=2,2

In [0]:
del X,Y

In [9]:
Y

NameError: ignored

In [23]:
%cd nsga-net/
!git pull https://github.com/DanielDimanov/nsga-net.git
%cd /content/

/content/nsga-net
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/DanielDimanov/nsga-net
 * branch            HEAD       -> FETCH_HEAD
Updating 62e5511..42c70ac
Fast-forward
 search/evolution_search.py | 1 +
 search/train_search.py     | 2 +-
 2 files changed, 2 insertions(+), 1 deletion(-)
/content


In [37]:
%cd nsga-net/

/content/nsga-net


In [42]:
%cd ..

/content


In [0]:
! mv -i nsga-net/ /content/gdrive/My\ Drive/MScProject/nsga-netv2

Usage: mv [OPTION]... [-T] SOURCE DEST
  or:  mv [OPTION]... SOURCE... DIRECTORY
  or:  mv [OPTION]... -t DIRECTORY SOURCE...
Rename SOURCE to DEST, or move SOURCE(s) to DIRECTORY.

Mandatory arguments to long options are mandatory for short options too.
      --backup[=CONTROL]       make a backup of each existing destination file
  -b                           like --backup but does not accept an argument
  -f, --force                  do not prompt before overwriting
  -i, --interactive            prompt before overwrite
  -n, --no-clobber             do not overwrite an existing file
If you specify more than one of -i, -f, -n, only the final one takes effect.
      --strip-trailing-slashes  remove any trailing slashes from each SOURCE
                                 argument
  -S, --suffix=SUFFIX          override the usual backup suffix
  -t, --target-directory=DIRECTORY  move all SOURCE arguments into DIRECTORY
  -T, --no-target-directory    treat DEST as a normal file
  -u, --u